In [1]:
import glob

import tensorflow as tf

from tensorflow.python.summary.summary_iterator import summary_iterator
from tensorflow.python.framework import tensor_util

from matplotlib import pyplot as plt
import yaml

import itertools

import numpy as np

import pandas as pd

In [2]:
metrics = ['accuracy_L1', 'accuracy_L12', 'loss', "I_XZ", "I_YZ"]

def event_file(path):
    files = glob.glob(f"{path}/*")
    return files[0]

def read_metrics(path):
    data = dict(zip(metrics,  [[] for x in range(len(metrics))]))
    
    for e in summary_iterator(event_file(path)):
        for v in e.summary.value:
            if v.tag in metrics:
                t = tensor_util.MakeNdarray(v.tensor)
                data[v.tag].append(float(t))
    return data


data = read_metrics("../artifacts-server/mnist--various-opts-M1/vdb-mnist--2019-11-25--17-46-49/logs/train");

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [257]:
def read_model_metrics(model_path, metric, average_n_last_epochs=5):
    with open(f"{model_path}/summary.yml", "r") as fh:
        summary = yaml.safe_load(fh)

    assert summary["metrics"]["train"]["loss"] > 0

        
    del summary["metrics"]
    del summary["epoch"]
    

    train_res = read_metrics(f"{model_path}/logs/train")
    test_res = read_metrics(f"{model_path}/logs/test")
    
    df = pd.DataFrame(list(range(1, len(train_res["accuracy_L1"])+1)), columns=["epoch"])
    
    for m in metrics:
        df[f"metrics:train:{m}"] = train_res[m]
        df[f"metrics:test:{m}"] = test_res[m]
        
    for k, v in summary.items():
        df[k] = v
        
    summary["_key"] = "M=%d::beta=%1.1e::strategy=%s" % (summary["M"], summary["beta"], summary["strategy"])

    df["metrics:train:error_L1"] =  1 - df["metrics:train:accuracy_L1"]
    df["metrics:test:error_L1"] =  1 - df["metrics:test:accuracy_L1"]
    df["metrics:train:error_L12"] =  1 - df["metrics:train:accuracy_L12"]
    df["metrics:test:error_L12"] =  1 - df["metrics:test:accuracy_L12"]
    summary["name"] = model_path.split("/")[-1]
    
    average_metric = df[f"metrics:{metric}"][(-average_n_last_epochs):].mean()
    summary[metric] = average_metric
    return summary

read_model_metrics(
    "../artifacts-server/mnist--various-opts-M1/vdb-mnist--2019-11-25--17-46-49",
    "test:accuracy_L12",
    average_n_last_epochs=5
)

{'M': 1,
 'beta': 0.001,
 'dataset': 'mnist',
 'lr': 0.0001,
 'model': 'vdb/e1:1024|e2:1024|z:256',
 'strategy': 'seq/d:5|e:1',
 '_key': 'M=1::beta=1.0e-03::strategy=seq/d:5|e:1',
 'name': 'vdb-mnist--2019-11-25--17-46-49',
 'test:accuracy_L12': 0.9848402380943299}

In [31]:
def get_models_from_dirs(dirs):
    files = []
    for d in dirs:
        files.extend(glob.glob(f"{d}/*/summary.yml"))
    
    files = list(map(lambda f: "/".join(f.split("/")[:-1]), files))
    print(f"Reading {','.join(dirs)}")
    print(f"we have {len(files)} models")
    return files

In [32]:
model_paths = get_models_from_dirs(["../artifacts-server/mnist--various-opts-M1"])

model_paths

Reading ../artifacts-server/mnist--various-opts-M1
we have 40 models


['../artifacts-server/mnist--various-opts-M1/vdb-mnist--2019-11-25--17-46-49',
 '../artifacts-server/mnist--various-opts-M1/vdb-mnist--2019-11-26--00-40-34',
 '../artifacts-server/mnist--various-opts-M1/vdb-mnist--2019-11-25--17-46-46',
 '../artifacts-server/mnist--various-opts-M1/vdb-mnist--2019-11-26--07-37-55',
 '../artifacts-server/mnist--various-opts-M1/vdb-mnist--2019-11-26--09-30-08-925070',
 '../artifacts-server/mnist--various-opts-M1/vdb-mnist--2019-11-26--12-10-41-823999',
 '../artifacts-server/mnist--various-opts-M1/vdb-mnist--2019-11-25--23-44-36',
 '../artifacts-server/mnist--various-opts-M1/vdb-mnist--2019-11-25--22-49-18',
 '../artifacts-server/mnist--various-opts-M1/vdb-mnist--2019-11-26--09-27-16-207667',
 '../artifacts-server/mnist--various-opts-M1/vdb-mnist--2019-11-26--12-10-36-984242',
 '../artifacts-server/mnist--various-opts-M1/vdb-mnist--2019-11-25--18-28-29',
 '../artifacts-server/mnist--various-opts-M1/vdb-mnist--2019-11-26--07-49-01',
 '../artifacts-server/mn

In [33]:
n_last = 5
metric = "test:error_L12"

In [34]:
def read_stats_from(dirs, metric, n=5):
    print(f"average over {n} last epochs")
    model_paths = get_models_from_dirs(dirs)
    
    df = pd.DataFrame([read_model_metrics(m, metric, average_n_last_epochs=n) for m in model_paths])
    return df

df_mnist_training_stats = read_stats_from(
    ["../artifacts-server/mnist--various-opts-M*"],
    metric=metric,
    n=n_last
)

average over 5 last epochs
Reading ../artifacts-server/mnist--various-opts-M*
we have 160 models


In [35]:
df_mnist_training_stats.groupby("_key")["M"].count()

_key
M=12::beta=1.0e-03::strategy=oneshot         5
M=12::beta=1.0e-03::strategy=seq/d:10|e:1    5
M=12::beta=1.0e-03::strategy=seq/d:20|e:1    5
M=12::beta=1.0e-03::strategy=seq/d:5|e:1     5
M=12::beta=1.0e-05::strategy=oneshot         5
M=12::beta=1.0e-05::strategy=seq/d:10|e:1    5
M=12::beta=1.0e-05::strategy=seq/d:20|e:1    5
M=12::beta=1.0e-05::strategy=seq/d:5|e:1     5
M=1::beta=1.0e-03::strategy=oneshot          5
M=1::beta=1.0e-03::strategy=seq/d:10|e:1     5
M=1::beta=1.0e-03::strategy=seq/d:20|e:1     5
M=1::beta=1.0e-03::strategy=seq/d:5|e:1      5
M=1::beta=1.0e-05::strategy=oneshot          5
M=1::beta=1.0e-05::strategy=seq/d:10|e:1     5
M=1::beta=1.0e-05::strategy=seq/d:20|e:1     5
M=1::beta=1.0e-05::strategy=seq/d:5|e:1      5
M=3::beta=1.0e-03::strategy=oneshot          5
M=3::beta=1.0e-03::strategy=seq/d:10|e:1     5
M=3::beta=1.0e-03::strategy=seq/d:20|e:1     5
M=3::beta=1.0e-03::strategy=seq/d:5|e:1      5
M=3::beta=1.0e-05::strategy=oneshot          5
M=3::bet

In [36]:
df_mnist_training_stats[:10]

,M,_key,beta,dataset,lr,model,strategy,test:error_L12
0,3,M=3::beta=1.0e-05::strategy=seq/d:20|e:1,0.00001,mnist,0.0001,vdb/e1:1024|e2:1024|z:256,seq/d:20|e:1,0.01440
1,3,M=3::beta=1.0e-05::strategy=oneshot,0.00001,mnist,0.0001,vdb/e1:1024|e2:1024|z:256,oneshot,0.01380
2,3,M=3::beta=1.0e-05::strategy=seq/d:5|e:1,0.00001,mnist,0.0001,vdb/e1:1024|e2:1024|z:256,seq/d:5|e:1,0.01434
3,3,M=3::beta=1.0e-03::strategy=seq/d:20|e:1,0.00100,mnist,0.0001,vdb/e1:1024|e2:1024|z:256,seq/d:20|e:1,0.01384
4,3,M=3::beta=1.0e-03::strategy=seq/d:20|e:1,0.00100,mnist,0.0001,vdb/e1:1024|e2:1024|z:256,seq/d:20|e:1,0.01452
5,3,M=3::beta=1.0e-03::strategy=seq/d:10|e:1,0.00100,mnist,0.0001,vdb/e1:1024|e2:1024|z:256,seq/d:10|e:1,0.01350
6,3,M=3::beta=1.0e-05::strategy=seq/d:10|e:1,0.00001,mnist,0.0001,vdb/e1:1024|e2:1024|z:256,seq/d:10|e:1,0.01358
7,3,M=3::beta=1.0e-05::strategy=seq/d:20|e:1,0.00001,mnist,0.0001,vdb/e1:1024|e2:1024|z:256,seq/d:20|e:1,0.01336
8,3,M=3::beta=1.0e-03::strategy=seq/d:20|e:1,0.00100,mnist,0.0001,vdb/e1:1024|e2:1024|z:256,seq/d:20|e:1,0.01456
9,3,M=3::beta=1.0e-03::strategy=seq/d:5|e:1,0.00100,mnist,0.0001,vdb/e1:1024|e2:1024|z:256,seq/d:5|e:1,0.01398


In [268]:
strategies = ['oneshot', 'seq/d:5|e:1', 'seq/d:10|e:1', 'seq/d:20|e:1']
strategies_mapping = dict(zip(strategies, range(len(strategies))))

def print_table(df, beta, metric, should_print=True):
    
    df = df[df.beta == beta]\
        .groupby(["strategy", "M"])[metric]\
        .agg({metric: ['mean', 'std']})\
        .reset_index()
        
    df["temp"] = df[metric][['mean', 'std']].apply(lambda x: "%.2f±%.4f\%%" % (x[0]*100, x[1]*100), axis=1) 

    
    df = df.drop(columns=[metric])\
        .rename(columns={"temp": f"{metric} ($\\beta=10^{{%d}}$)" % np.log10(beta)}) \
        .pivot(index="strategy", columns="M")
    
    if should_print:
        print(df.to_latex(column_format="rcccc", escape=False))
    
    return df

    
print_table(df_mnist_training_stats, 0.001, metric)

\begin{tabular}{rcccc}
\toprule
{} & \multicolumn{4}{l}{test:error_L12 ($\beta=10^{-3}$)} \\
{} \\
M &                               1  &             3  &             6  &             12 \\
strategy     &                                  &                &                &                \\
\midrule
oneshot      &                    1.52±0.0746\% &  1.48±0.0491\% &  1.42±0.0652\% &  1.41±0.0767\% \\
seq/d:10|e:1 &                    1.46±0.0276\% &  1.45±0.0800\% &  1.38±0.0498\% &  1.40±0.0513\% \\
seq/d:20|e:1 &                    1.47±0.0686\% &  1.42±0.0499\% &  1.42±0.0540\% &  1.43±0.0687\% \\
seq/d:5|e:1  &                    1.48±0.0355\% &  1.42±0.0540\% &  1.42±0.0471\% &  1.45±0.0428\% \\
\bottomrule
\end{tabular}



/Users/heytitle/projects/venv-python3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


test:error_L12 ($\beta=10^{-3}$)                                \
                                                                              
M                                          1              3              6    
strategy                                                                      
oneshot                         1.52±0.0746\%  1.48±0.0491\%  1.42±0.0652\%   
seq/d:10|e:1                    1.46±0.0276\%  1.45±0.0800\%  1.38±0.0498\%   
seq/d:20|e:1                    1.47±0.0686\%  1.42±0.0499\%  1.42±0.0540\%   
seq/d:5|e:1                     1.48±0.0355\%  1.42±0.0540\%  1.42±0.0471\%   

                             
                             
M                        12  
strategy                     
oneshot       1.41±0.0767\%  
seq/d:10|e:1  1.40±0.0513\%  
seq/d:20|e:1  1.43±0.0687\%  
seq/d:5|e:1   1.45±0.0428\%

In [38]:
# print_table(df_mnist_training_stats, 0.00001, metric)

In [39]:
df_mnist_training_stats_lr_decay = read_stats_from(
    ["../artifacts-server/mnist--lr-decay-various-opts-*"],
    metric=metric,
    n=n_last
)

average over 5 last epochs
Reading ../artifacts-server/mnist--lr-decay-various-opts-*
we have 16 models


In [40]:
print_table(df_mnist_training_stats_lr_decay, 0.001, metric, should_print=False)

/Users/heytitle/projects/venv-python3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


test:error_L12 ($\beta=10^{-3}$ and averaged over 5 epochs)  \
                                                                           
M                                                                      1   
strategy                                                                   
oneshot                                           1.40±0.0171\%            
seq/d:10|e:1                                      1.37±0.0727\%            
seq/d:20|e:1                                      1.32±0.0301\%            
seq/d:5|e:1                                       1.39±0.0857\%            

                          
                          
M                      6  
strategy                  
oneshot       1.41±nan\%  
seq/d:10|e:1  1.44±nan\%  
seq/d:20|e:1  1.46±nan\%  
seq/d:5|e:1   1.40±nan\%

In [41]:
# print_table(df_mnist_training_stats_lr_decay, 0.00001, metric, should_print=False)

In [42]:
df_mnist_training_stats_lr_decay_algo2 = read_stats_from(
    ["../artifacts-server/train_algo2-/*", "../artifacts-server/train-/*"],
    metric=metric,
    n=n_last
)

average over 5 last epochs
Reading ../artifacts-server/train_algo2-/*,../artifacts-server/train-/*
we have 81 models


In [43]:
df_mnist_training_stats_lr_decay_algo2.groupby("_key")["M"].count()

_key
M=12::beta=1.0e-03::strategy=alt/e:10|d:1    3
M=12::beta=1.0e-03::strategy=alt/e:20|d:1    3
M=12::beta=1.0e-03::strategy=alt/e:5|d:1     3
M=12::beta=1.0e-03::strategy=oneshot         3
M=12::beta=1.0e-03::strategy=seq/d:10|e:1    3
M=12::beta=1.0e-03::strategy=seq/d:20|e:1    3
M=12::beta=1.0e-03::strategy=seq/d:5|e:1     3
M=1::beta=1.0e-03::strategy=alt/e:10|d:1     3
M=1::beta=1.0e-03::strategy=alt/e:20|d:1     3
M=1::beta=1.0e-03::strategy=alt/e:5|d:1      3
M=1::beta=1.0e-03::strategy=oneshot          5
M=1::beta=1.0e-03::strategy=seq/d:10|e:1     3
M=1::beta=1.0e-03::strategy=seq/d:20|e:1     3
M=1::beta=1.0e-03::strategy=seq/d:5|e:1      3
M=3::beta=1.0e-03::strategy=alt/e:10|d:1     3
M=3::beta=1.0e-03::strategy=alt/e:20|d:1     3
M=3::beta=1.0e-03::strategy=alt/e:5|d:1      3
M=3::beta=1.0e-03::strategy=oneshot          2
M=3::beta=1.0e-03::strategy=seq/d:10|e:1     2
M=3::beta=1.0e-03::strategy=seq/d:20|e:1     1
M=3::beta=1.0e-03::strategy=seq/d:5|e:1      2
M=6::bet

In [44]:
print_table(df_mnist_training_stats_lr_decay_algo2, 0.001, metric, should_print=False)

/Users/heytitle/projects/venv-python3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


test:error_L12 ($\beta=10^{-3}$ and averaged over 5 epochs)  \
                                                                           
M                                                                     1    
strategy                                                                   
alt/e:10|d:1                                      1.31±0.0324\%            
alt/e:20|d:1                                      1.31±0.0673\%            
alt/e:5|d:1                                       1.27±0.0142\%            
oneshot                                           1.30±0.0192\%            
seq/d:10|e:1                                      1.30±0.0620\%            
seq/d:20|e:1                                      1.29±0.0500\%            
seq/d:5|e:1                                       1.31±0.0319\%            

                                                           
                                                           
M                        3              6              12  
strategy                                                   
alt/e:10|d:1  1.32±0.0266\%  1.32±0.0866\%  1.35±0.0480\%  
alt/e:20|d:1  1.34±0.0244\%  1.37±0.0140\%  1.32±0.0170\%  
alt/e:5|d:1   1.27±0.0391\%  1.31±0.0570\%  1.34±0.0142\%  
oneshot       1.36±0.0170\%  1.28±0.0179\%  1.34±0.0746\%  
seq/d:10|e:1  1.33±0.0651\%  1.34±0.1280\%  1.34±0.0588\%  
seq/d:20|e:1     1.30±nan\%  1.29±0.0722\%  1.34±0.0647\%  
seq/d:5|e:1   1.27±0.0354\%  1.38±0.0530\%  1.32±0.0383\%

In [45]:
print_table(df_mnist_training_stats_lr_decay_algo2, 0.00001, metric, should_print=False)

/Users/heytitle/projects/venv-python3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [ ]:
# df_mnist_training_stats_lr_decay.groupby("_key")["M"].count()

In [ ]:
df = read_stats_from(
    ["../artifacts-server/train-/fix-normalised*"],
    metric=metric,
    n=n_last
)

In [ ]:
print_table(df, 0.001, metric, should_print=False)

In [ ]:
df2 = read_stats_from(
    ["../artifacts-server/train-/cherry-pick-fix-normalised*"],
    metric=metric,
    n=n_last
)

In [ ]:
print_table(df2, 0.001, metric, should_print=False)

In [ ]:
df_data = read_stats_from(
    ["../artifacts-server/train-/*", "../artifacts-server/train_algo2-/*"],
    metric=metric,
    n=5
)
print_table( df_data, 0.001, metric, should_print=False)

In [ ]:
df_data.groupby("_key")["M"].count()

In [ ]:
df_data_wo_lr_decay = read_stats_from(
    ["../artifacts-server/experiment-mnist-wo-lr-decay/*"],
    metric=metric,
    n=5
)

In [ ]:
df_data_wo_lr_decay.groupby("_key")["M"].count()

In [ ]:
print_table(df_data_wo_lr_decay, 0.001, metric, should_print=True)

In [ ]:
print_table(df_data_wo_lr_decay, 0.00001, metric, should_print=True)

In [ ]:
df_data_wo_lr_decay_replicating_tab4 = read_stats_from(
    ["../artifacts-server/experiment-mnist-wo-lr-decay-replicating-table-4/*"],
    metric=metric,
    n=3
)

In [ ]:
df_data_wo_lr_decay_replicating_tab4.groupby("_key")["M"].count()

In [ ]:
print_table(df_data_wo_lr_decay_replicating_tab4, 0.001, metric, should_print=True)

In [ ]:
df_data_global_lr_decay = read_stats_from(
    ["../artifacts-server/experiment-mnist-global-lr-decay/*"],
    metric=metric,
    n=5
)

df_data_global_lr_decay.groupby("_key")["M"].count()

In [ ]:
print_table(df_data_global_lr_decay, 0.001, metric, should_print=False)

In [ ]:
df_data_vib_classloss = read_stats_from(
    ["../artifacts-server/experiment-vib-classloss/*"],
    metric=metric,
    n=3
)

df_data_vib_classloss.groupby("_key")["M"].count()

In [ ]:
print_table(df_data_vib_classloss, 0.001, metric, should_print=True)

# Resnet20 MPG Server

In [166]:
df_data_resnet_mpg = read_stats_from(
    ["../artifacts-server/mpg-server/experiment-resnet20-cifar10/*"],
    metric=metric,
    n=3
)

df_data_resnet_mpg.groupby("_key")["M"].count()

average over 3 last epochs
Reading ../artifacts-server/mpg-server/experiment-resnet20-cifar10/*
we have 60 models


_key
M=12::beta=1.0e-02::strategy=alt/e:10|d:1    4
M=12::beta=1.0e-02::strategy=oneshot         4
M=12::beta=1.0e-02::strategy=seq/d:10|e:1    4
M=12::beta=1.0e-03::strategy=oneshot         4
M=12::beta=1.0e-03::strategy=seq/d:10|e:1    4
M=1::beta=1.0e-02::strategy=alt/e:10|d:1     4
M=1::beta=1.0e-02::strategy=oneshot          4
M=1::beta=1.0e-02::strategy=seq/d:10|e:1     4
M=1::beta=1.0e-03::strategy=oneshot          4
M=1::beta=1.0e-03::strategy=seq/d:10|e:1     4
M=6::beta=1.0e-02::strategy=alt/e:10|d:1     4
M=6::beta=1.0e-02::strategy=oneshot          4
M=6::beta=1.0e-02::strategy=seq/d:10|e:1     4
M=6::beta=1.0e-03::strategy=oneshot          4
M=6::beta=1.0e-03::strategy=seq/d:10|e:1     4
Name: M, dtype: int64

In [167]:
print_table(df_data_resnet_mpg, 0.01, metric, should_print=False)

/Users/heytitle/projects/venv-python3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


test:error_L12 ($\beta=10^{-2}$ and averaged over 5 epochs)  \
                                                                           
M                                                                     1    
strategy                                                                   
alt/e:10|d:1                                     26.48±0.2818\%            
oneshot                                          27.00±0.7832\%            
seq/d:10|e:1                                     26.52±0.7592\%            

                                              
                                              
M                         6               12  
strategy                                      
alt/e:10|d:1  26.22±0.9204\%  26.34±1.0436\%  
oneshot       26.58±1.0938\%  25.81±0.7480\%  
seq/d:10|e:1  26.00±0.7002\%  26.05±1.1479\%

In [169]:
print_table(df_data_resnet_mpg, 0.001, metric, should_print=False)

/Users/heytitle/projects/venv-python3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


test:error_L12 ($\beta=10^{-3}$ and averaged over 5 epochs)  \
                                                                           
M                                                                     1    
strategy                                                                   
oneshot                                          26.78±0.9641\%            
seq/d:10|e:1                                     26.03±0.4112\%            

                                              
                                              
M                         6               12  
strategy                                      
oneshot       26.92±1.1053\%  26.91±0.9535\%  
seq/d:10|e:1  25.86±1.0348\%  29.71±9.6670\%

In [170]:
df_data_resnet_mpg[ (df_data_resnet_mpg.beta == 0.001) & (df_data_resnet_mpg.strategy == "oneshot")]

,M,_key,algorithm,beta,dataset,lr,model,strategy,test:error_L12
8,12,M=12::beta=1.0e-03::strategy=oneshot,NaN,0.001,cifar10,0.001,resnet20/z:15,oneshot,0.264267
12,12,M=12::beta=1.0e-03::strategy=oneshot,NaN,0.001,cifar10,0.001,resnet20/z:15,oneshot,0.265200
14,12,M=12::beta=1.0e-03::strategy=oneshot,NaN,0.001,cifar10,0.001,resnet20/z:15,oneshot,0.263567
17,12,M=12::beta=1.0e-03::strategy=oneshot,NaN,0.001,cifar10,0.001,resnet20/z:15,oneshot,0.283367
24,1,M=1::beta=1.0e-03::strategy=oneshot,NaN,0.001,cifar10,0.001,resnet20/z:15,oneshot,0.258600
30,1,M=1::beta=1.0e-03::strategy=oneshot,NaN,0.001,cifar10,0.001,resnet20/z:15,oneshot,0.262867
32,1,M=1::beta=1.0e-03::strategy=oneshot,NaN,0.001,cifar10,0.001,resnet20/z:15,oneshot,0.280800
37,1,M=1::beta=1.0e-03::strategy=oneshot,NaN,0.001,cifar10,0.001,resnet20/z:15,oneshot,0.268733
40,6,M=6::beta=1.0e-03::strategy=oneshot,NaN,0.001,cifar10,0.001,resnet20/z:15,oneshot,0.259933
42,6,M=6::beta=1.0e-03::strategy=oneshot,NaN,0.001,cifar10,0.001,resnet20/z:15,oneshot,0.259400


In [165]:
np.random.choice(range(10), 5)

array([6, 2, 6, 3, 7])

In [284]:
df_data_resnet_cifar10_10k = read_stats_from(
    ["../artifacts-server/mpg-server/experiment-cifar-subsets/cifar10-10k-*"],
    metric="test:error_L12",
    n=3
)

df_data_resnet_cifar10_10k.groupby("_key")["M"].count()

average over 3 last epochs
Reading ../artifacts-server/mpg-server/experiment-cifar-subsets/cifar10-10k-*
we have 10 models


_key
M=12::beta=1.0e-03::strategy=alt/e:10|d:1    2
M=1::beta=1.0e-03::strategy=alt/e:10|d:1     1
M=1::beta=1.0e-03::strategy=oneshot          4
M=6::beta=1.0e-03::strategy=alt/e:10|d:1     1
M=6::beta=1.0e-03::strategy=oneshot          2
Name: M, dtype: int64

In [285]:
print_table(df_data_resnet_cifar10_10k, 0.001, "test:error_L12", should_print=False)

/Users/heytitle/projects/venv-python3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


test:error_L12 ($\beta=10^{-3}$)                                
                                                                             
M                                          1               6               12
strategy                                                                     
alt/e:10|d:1                      49.24±nan\%     53.83±nan\%  55.96±5.2114\%
oneshot                        55.00±2.5321\%  54.94±0.2475\%             NaN

In [282]:
df_data_resnet_cifar10_40k = read_stats_from(
    ["../artifacts-server/mpg-server/experiment-cifar-subsets/cifar10-40k-*"],
    metric="test:error_L12",
    n=3
)

df_data_resnet_cifar10_40k.groupby("_key")["M"].count()

average over 3 last epochs
Reading ../artifacts-server/mpg-server/experiment-cifar-subsets/cifar10-40k-*
we have 10 models


_key
M=12::beta=1.0e-03::strategy=alt/e:10|d:1    3
M=1::beta=1.0e-03::strategy=alt/e:10|d:1     4
M=6::beta=1.0e-03::strategy=alt/e:10|d:1     3
Name: M, dtype: int64

In [283]:
print_table(df_data_resnet_cifar10_40k, 0.001, "test:error_L12", should_print=False)

/Users/heytitle/projects/venv-python3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


test:error_L12 ($\beta=10^{-3}$)                                
                                                                             
M                                          1               6               12
strategy                                                                     
alt/e:10|d:1                   28.31±1.3237\%  33.65±3.3138\%  31.69±4.9326\%

In [273]:
df_data_resnet_fashion_mnist_40k = read_stats_from(
    ["../artifacts-server/mpg-server/experiment-fashion_mnist-subsets/fashion_mnist-40k-*"],
    metric="test:error_L12",
    n=3
)

df_data_resnet_fashion_mnist_40k.groupby("_key")["M"].count()

average over 3 last epochs
Reading ../artifacts-server/mpg-server/experiment-fashion_mnist-subsets/fashion_mnist-40k-*
we have 32 models


_key
M=12::beta=1.0e-03::strategy=alt/e:10|d:1    3
M=12::beta=1.0e-03::strategy=oneshot         4
M=12::beta=1.0e-03::strategy=seq/d:10|e:1    4
M=1::beta=1.0e-03::strategy=alt/e:10|d:1     4
M=1::beta=1.0e-03::strategy=oneshot          4
M=1::beta=1.0e-03::strategy=seq/d:10|e:1     4
M=6::beta=1.0e-03::strategy=alt/e:10|d:1     4
M=6::beta=1.0e-03::strategy=oneshot          3
M=6::beta=1.0e-03::strategy=seq/d:10|e:1     2
Name: M, dtype: int64

In [274]:
print_table(df_data_resnet_fashion_mnist_40k, 0.001, "test:error_L12", should_print=False)

/Users/heytitle/projects/venv-python3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


test:error_L12 ($\beta=10^{-3}$)                                
                                                                             
M                                          1              6                12
strategy                                                                     
alt/e:10|d:1                    8.83±0.2011\%  9.37±0.4735\%  35.78±46.9452\%
oneshot                         9.10±0.2765\%  8.64±0.4972\%    8.93±0.2755\%
seq/d:10|e:1                    9.10±0.1124\%  9.70±0.7590\%   10.02±1.3405\%

In [277]:
df_data_resnet_fashion_mnist_10k = read_stats_from(
    ["../artifacts-server/mpg-server/experiment-fashion_mnist-subsets/fashion_mnist-10k-*"],
    metric="test:error_L12",
    n=3
)

df_data_resnet_fashion_mnist_10k.groupby("_key")["M"].count()

average over 3 last epochs
Reading ../artifacts-server/mpg-server/experiment-fashion_mnist-subsets/fashion_mnist-10k-*
we have 17 models


_key
M=12::beta=1.0e-03::strategy=alt/e:10|d:1    1
M=12::beta=1.0e-03::strategy=oneshot         1
M=12::beta=1.0e-03::strategy=seq/d:10|e:1    3
M=1::beta=1.0e-03::strategy=alt/e:10|d:1     3
M=1::beta=1.0e-03::strategy=oneshot          3
M=1::beta=1.0e-03::strategy=seq/d:10|e:1     2
M=6::beta=1.0e-03::strategy=alt/e:10|d:1     1
M=6::beta=1.0e-03::strategy=oneshot          1
M=6::beta=1.0e-03::strategy=seq/d:10|e:1     2
Name: M, dtype: int64

In [276]:
print_table(df_data_resnet_fashion_mnist_10k, 0.001, "test:error_L12", should_print=False)

/Users/heytitle/projects/venv-python3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


test:error_L12 ($\beta=10^{-3}$)                                
                                                                             
M                                          1               6               12
strategy                                                                     
alt/e:10|d:1                   15.17±0.4373\%     13.94±nan\%     13.83±nan\%
oneshot                        13.60±1.7068\%     14.03±nan\%     14.91±nan\%
seq/d:10|e:1                   13.78±0.1626\%  13.76±1.5698\%  14.66±0.4802\%

In [272]:
df_data_resnet_fashion_mnist_10k

,M,_key,beta,dataset,lr,model,name,strategy,test:error_L12
0,1,M=1::beta=1.0e-03::strategy=alt/e:10|d:1,0.001,fashion_mnist-10k,0.001,resnet20/z:15,resnet20-fashion_mnist-10k--2019-12-18--08-17-...,alt/e:10|d:1,0.148033
1,1,M=1::beta=1.0e-03::strategy=alt/e:10|d:1,0.001,fashion_mnist-10k,0.001,resnet20/z:15,resnet20-fashion_mnist-10k--2019-12-18--08-02-...,alt/e:10|d:1,0.156533
2,1,M=1::beta=1.0e-03::strategy=oneshot,0.001,fashion_mnist-10k,0.001,resnet20/z:15,resnet20-fashion_mnist-10k--2019-12-18--07-56-...,oneshot,0.155667
3,1,M=1::beta=1.0e-03::strategy=seq/d:10|e:1,0.001,fashion_mnist-10k,0.001,resnet20/z:15,resnet20-fashion_mnist-10k--2019-12-18--08-11-...,seq/d:10|e:1,0.136633
4,1,M=1::beta=1.0e-03::strategy=oneshot,0.001,fashion_mnist-10k,0.001,resnet20/z:15,resnet20-fashion_mnist-10k--2019-12-18--08-04-...,oneshot,0.125600
5,1,M=1::beta=1.0e-03::strategy=alt/e:10|d:1,0.001,fashion_mnist-10k,0.001,resnet20/z:15,resnet20-fashion_mnist-10k--2019-12-18--08-11-...,alt/e:10|d:1,0.150500
6,1,M=1::beta=1.0e-03::strategy=seq/d:10|e:1,0.001,fashion_mnist-10k,0.001,resnet20/z:15,resnet20-fashion_mnist-10k--2019-12-18--07-58-...,seq/d:10|e:1,0.138933
7,1,M=1::beta=1.0e-03::strategy=oneshot,0.001,fashion_mnist-10k,0.001,resnet20/z:15,resnet20-fashion_mnist-10k--2019-12-18--08-18-...,oneshot,0.126633
8,6,M=6::beta=1.0e-03::strategy=seq/d:10|e:1,0.001,fashion_mnist-10k,0.001,resnet20/z:15,resnet20-fashion_mnist-10k--2019-12-18--08-42-...,seq/d:10|e:1,0.148733
9,6,M=6::beta=1.0e-03::strategy=seq/d:10|e:1,0.001,fashion_mnist-10k,0.001,resnet20/z:15,resnet20-fashion_mnist-10k--2019-12-18--08-36-...,seq/d:10|e:1,0.126533


# VDB MLP Robustness

In [307]:
df_data_vdb_mnist_robustness = read_stats_from(
    ["../artifacts-server/hydra/experiment-mnist-256d-test-robustness/*"],
    metric="test:error_L12",
    n=5
)

df_data_vdb_mnist_robustness.groupby("_key")["M"].count()

average over 5 last epochs
Reading ../artifacts-server/hydra/experiment-mnist-256d-test-robustness/*
we have 112 models


_key
M=12::beta=1.0e-03::strategy=algo1/d:10|e:1    4
M=12::beta=1.0e-03::strategy=algo1/d:20|e:1    4
M=12::beta=1.0e-03::strategy=algo1/d:5|e:1     4
M=12::beta=1.0e-03::strategy=algo2/k:10        4
M=12::beta=1.0e-03::strategy=algo2/k:20        4
M=12::beta=1.0e-03::strategy=algo2/k:5         4
M=12::beta=1.0e-03::strategy=oneshot           4
M=18::beta=1.0e-03::strategy=algo1/d:10|e:1    4
M=18::beta=1.0e-03::strategy=algo1/d:20|e:1    4
M=18::beta=1.0e-03::strategy=algo1/d:5|e:1     4
M=18::beta=1.0e-03::strategy=algo2/k:10        4
M=18::beta=1.0e-03::strategy=algo2/k:20        4
M=18::beta=1.0e-03::strategy=algo2/k:5         4
M=18::beta=1.0e-03::strategy=oneshot           4
M=1::beta=1.0e-03::strategy=algo1/d:10|e:1     4
M=1::beta=1.0e-03::strategy=algo1/d:20|e:1     4
M=1::beta=1.0e-03::strategy=algo1/d:5|e:1      4
M=1::beta=1.0e-03::strategy=algo2/k:10         4
M=1::beta=1.0e-03::strategy=algo2/k:20         4
M=1::beta=1.0e-03::strategy=algo2/k:5          4
M=1::beta=1.0e-

In [309]:
print_table(df_data_vdb_mnist_robustness, 0.001, "test:error_L12", should_print=True)

\begin{tabular}{rcccc}
\toprule
{} & \multicolumn{4}{l}{test:error_L12 ($\beta=10^{-3}$)} \\
{} \\
M &                               1  &             6  &             12 &             18 \\
strategy       &                                  &                &                &                \\
\midrule
algo1/d:10|e:1 &                    1.48±0.0590\% &  1.41±0.0303\% &  1.43±0.1309\% &  1.44±0.1264\% \\
algo1/d:20|e:1 &                    1.39±0.0388\% &  1.43±0.0681\% &  1.38±0.0708\% &  1.39±0.0126\% \\
algo1/d:5|e:1  &                    1.46±0.0519\% &  1.38±0.0245\% &  1.45±0.0445\% &  1.34±0.0553\% \\
algo2/k:10     &                    1.47±0.0605\% &  1.41±0.0516\% &  1.36±0.0339\% &  1.41±0.0496\% \\
algo2/k:20     &                    1.45±0.0473\% &  1.38±0.0462\% &  1.41±0.0541\% &  1.36±0.0580\% \\
algo2/k:5      &                    1.49±0.0399\% &  1.42±0.0629\% &  1.39±0.0656\% &  1.42±0.0503\% \\
oneshot        &                    1.50±0.0727\% &  1.42±0.0486\% &  1.4

/Users/heytitle/projects/venv-python3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


test:error_L12 ($\beta=10^{-3}$)                                \
                                                                                
M                                            1              6              12   
strategy                                                                        
algo1/d:10|e:1                    1.48±0.0590\%  1.41±0.0303\%  1.43±0.1309\%   
algo1/d:20|e:1                    1.39±0.0388\%  1.43±0.0681\%  1.38±0.0708\%   
algo1/d:5|e:1                     1.46±0.0519\%  1.38±0.0245\%  1.45±0.0445\%   
algo2/k:10                        1.47±0.0605\%  1.41±0.0516\%  1.36±0.0339\%   
algo2/k:20                        1.45±0.0473\%  1.38±0.0462\%  1.41±0.0541\%   
algo2/k:5                         1.49±0.0399\%  1.42±0.0629\%  1.39±0.0656\%   
oneshot                           1.50±0.0727\%  1.42±0.0486\%  1.40±0.0545\%   

                               
                               
M                          18  
strategy                       
algo1/d:10|e:1  1.44±0.1264\%  
algo1/d:20|e:1  1.39±0.0126\%  
algo1/d:5|e:1   1.34±0.0553\%  
algo2/k:10      1.41±0.0496\%  
algo2/k:20      1.36±0.0580\%  
algo2/k:5       1.42±0.0503\%  
oneshot         1.41±0.0727\%

In [301]:
df_data_vdb_mnist_robustness

,M,_key,beta,class_loss,cov_type,dataset,lr,model,name,strategy,test:error_L12
0,12,M=12::beta=1.0e-03::strategy=algo2/k:20,0.001,vdb,diag,mnist,0.0001,mlp/e1:1024|e2:1024|z:256,mlp-vdb-diag-mnist--2019-12-21--21-43-34-051545,algo2/k:20,0.01398
1,12,M=12::beta=1.0e-03::strategy=algo2/k:10,0.001,vdb,diag,mnist,0.0001,mlp/e1:1024|e2:1024|z:256,mlp-vdb-diag-mnist--2019-12-21--21-40-32-703875,algo2/k:10,0.01326
2,12,M=12::beta=1.0e-03::strategy=algo2/k:20,0.001,vdb,diag,mnist,0.0001,mlp/e1:1024|e2:1024|z:256,mlp-vdb-diag-mnist--2019-12-21--22-51-51-234667,algo2/k:20,0.01474
3,12,M=12::beta=1.0e-03::strategy=algo1/d:5|e:1,0.001,vdb,diag,mnist,0.0001,mlp/e1:1024|e2:1024|z:256,mlp-vdb-diag-mnist--2019-12-21--21-31-04-697033,algo1/d:5|e:1,0.01508
4,12,M=12::beta=1.0e-03::strategy=algo2/k:10,0.001,vdb,diag,mnist,0.0001,mlp/e1:1024|e2:1024|z:256,mlp-vdb-diag-mnist--2019-12-21--22-50-51-979093,algo2/k:10,0.01404
5,12,M=12::beta=1.0e-03::strategy=algo1/d:5|e:1,0.001,vdb,diag,mnist,0.0001,mlp/e1:1024|e2:1024|z:256,mlp-vdb-diag-mnist--2019-12-21--22-55-58-931759,algo1/d:5|e:1,0.01404
6,12,M=12::beta=1.0e-03::strategy=algo2/k:5,0.001,vdb,diag,mnist,0.0001,mlp/e1:1024|e2:1024|z:256,mlp-vdb-diag-mnist--2019-12-21--21-40-25-344431,algo2/k:5,0.01344
7,12,M=12::beta=1.0e-03::strategy=algo1/d:10|e:1,0.001,vdb,diag,mnist,0.0001,mlp/e1:1024|e2:1024|z:256,mlp-vdb-diag-mnist--2019-12-21--21-34-32-159398,algo1/d:10|e:1,0.01356
8,12,M=12::beta=1.0e-03::strategy=oneshot,0.001,vdb,diag,mnist,0.0001,mlp/e1:1024|e2:1024|z:256,mlp-vdb-diag-mnist--2019-12-21--22-55-44-715543,oneshot,0.01430
9,12,M=12::beta=1.0e-03::strategy=algo1/d:20|e:1,0.001,vdb,diag,mnist,0.0001,mlp/e1:1024|e2:1024|z:256,mlp-vdb-diag-mnist--2019-12-21--21-39-27-758087,algo1/d:20|e:1,0.01356


## VIB MLP

In [313]:
df_data_vib_mnist_robustness = read_stats_from(
    ["../artifacts-server/hydra/experiment-mnist-256d-test-robustness-vib/*"],
    metric="test:error_L12",
    n=5
)

df_data_vib_mnist_robustness.groupby("_key")["M"].count()

average over 5 last epochs
Reading ../artifacts-server/hydra/experiment-mnist-256d-test-robustness-vib/*
we have 12 models


_key
M=12::beta=1.0e-03::strategy=oneshot    4
M=18::beta=1.0e-03::strategy=oneshot    4
M=6::beta=1.0e-03::strategy=oneshot     4
Name: M, dtype: int64

In [315]:
print_table(df_data_vib_mnist_robustness, 0.001, "test:error_L12", should_print=True)

\begin{tabular}{rcccc}
\toprule
{} & \multicolumn{3}{l}{test:error_L12 ($\beta=10^{-3}$)} \\
{} \\
M &                               6  &             12 &             18 \\
strategy &                                  &                &                \\
\midrule
oneshot  &                    1.45±0.0958\% &  1.33±0.0417\% &  1.35±0.0820\% \\
\bottomrule
\end{tabular}



/Users/heytitle/projects/venv-python3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


test:error_L12 ($\beta=10^{-3}$)                              
                                                                       
M                                      6              12             18
strategy                                                               
oneshot                     1.45±0.0958\%  1.33±0.0417\%  1.35±0.0820\%